In [ ]:
import os
import sys
proj_dir = os.path.dirname(os.path.realpath(os.getcwd()))
sys.path.insert(0, proj_dir)

import pandas as pd
from dataset.lamah import LamaH

import numpy as np
import networkx as nx

from bokeh.io import output_file, show
from bokeh.plotting import figure, from_networkx
from bokeh.models import GraphRenderer, Circle, StaticLayoutProvider
from bokeh.palettes import Spectral4
from bokeh.models import (BoxSelectTool, BoxZoomTool, ResetTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, TapTool, Range1d)

import pydot
from networkx.drawing.nx_pydot import graphviz_layout

## Load graph

In [ ]:
lamah = LamaH(root=os.path.join(proj_dir, 'data/'))
ts_qobs_df = lamah.df
ts_qobs_df.columns = ts_qobs_df.columns.droplevel(1)

In [ ]:
gauge_attribs, _ = lamah.load_plotting_data()
nodes = list(range(len(lamah.nodes)))
attribs = [dict(rID=n,
                gID=i,
                n_name=gauge_attribs.loc[n, 'name'],
                area=gauge_attribs.loc[n, 'area_gov'],
                elev=gauge_attribs.loc[n, 'elev'],
                start=gauge_attribs.loc[n, 'obsbeg_day'],
                impact_type=gauge_attribs.loc[n, 'typimpact'].replace(',', ', '),
                impact_deg=gauge_attribs.loc[n, 'degimpact']) for i,n in enumerate(lamah.nodes)]
nodes_attribs = list(zip(nodes, attribs))

######

g = nx.Graph()

g.add_nodes_from(nodes_attribs)
edge_index, _ = lamah.get_connectivity(method='binary',
                                       layout='edge_index')
g.add_edges_from(edge_index.T)

## Plot Graph

In [ ]:
plot = figure(title='LamaH-CE Network', x_range=Range1d(4.150e+6, 5.000e+6), y_range=Range1d(2.500e+6, 3.100e+6),
              width=1800, height=950)

node_hover_tool = HoverTool(tooltips=[('Real ID', '@rID'),
                                      ('Graph ID', '@gID'),
                                      ('Name', '@n_name'),
                                      ('Area (km^2)', '@area'),
                                      ('Elevation (m a.s.l.)', '@elev'),
                                      ('Obs start (year)', '@start'),
                                      ('Impact type', '@impact_type'),
                                      ('Impact deg', '@impact_deg')])
plot.add_tools(node_hover_tool, TapTool())

### Create Graph Renderer from nextworkx (pydot to layout nodes)

In [ ]:
graph_renderer = from_networkx(g, graphviz_layout(g, prog='dot'), scale=1, center=(0,0)) #dot

### Use geographic layout

In [ ]:
# create lists of x- and y-coordinates
x = gauge_attribs.loc[:, 'lon']
y = gauge_attribs.loc[:, 'lat']

# convert the ``x`` and ``y`` lists into a dictionary of 2D-coordinates
# and assign each entry to a node on the ``node_indices`` list
graph_layout = dict(zip(nodes, zip(x, y)))

# use the provider model to supply coourdinates to the graph
graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

### Complete plotting procedure

In [ ]:
graph_renderer.node_renderer.glyph = Circle(size=10, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=10, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=10, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color='#CCCCCC', line_alpha=0.8, line_width=2)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=2)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=2)

graph_renderer.selection_policy = NodesAndLinkedEdges()
# graph_renderer.inspection_policy = EdgesAndLinkedNodes()


plot.renderers.append(graph_renderer)

output_file(os.path.join(proj_dir, 'data/LamaH-CE/lamah_network.html'))
show(plot)